# GPT

GPT(Generative Pretrained Transformer)はTransformerベースの言語モデルです。ChatGPT などの生成系 AI アプリケーションの基礎となっている人工知能 (AI) の重要な新技術です。GPT モデルにより、アプリケーションは人間のようにテキストやコンテンツ (画像、音楽など) を作成したり、会話形式で質問に答えたりすることができます。さまざまな業界の組織が、Q&A ボット、テキスト要約、コンテンツ生成、検索に GPT モデルと生成系 AI を使用しています。

GPTはOpenAIによって定期的に新しいバージョンが公開されていますが、ここではGPT-2について解説します。

![](./Figure/gpt_history.png)

![](./Figure/gpt2.png)


## 入力表現

GPTの入力は、入力トークン列に対応するトークン埋め込み$e_{w_i}$と位置埋め込む$p_i$を加算した埋め込み列です。

$$
x_i=e_{w_i}+p_i
$$

## 事前学習

GPTの事前学習タスクは、入力されたトークン列の次のトークンを予測することです。ここで、GPTはデコーダ構成のTransformerを用います。

![](./Figure/gpt_input_representation.png)

```{margin}
GPTはオリジナルのTransformerにいくつかの改装を行いました。
```

学習に用いるトークン列$w_1,w_2,...,w_N$におけるのトークン$w_i$を予測することを考えます。GPTでは、予測確率を使った負の対数尤度を損失関数として事前学習を行います。

$$
\zeta(\theta)=- \sum_i log P(w_i|w_{i-K},....w_{i-1},\theta)
$$

ここで、$\theta$はモデルに含まれるすべてのパラメータを表します。

学習時にはMasked Self-Attention機構が導入され、入力トークン列の各位置において次のトークンを予測して学習が行われます。


![](./Figure/gpt_decoder.png)


## ファインチューング

GPTの事前学習済みモデルに、下流タスクに合わせて変換するためのヘッドを追加し、下流タスクのデータセットを用いてモデル全体を調整します。

GPTは下流タスクを解く際、特殊トークンを用いて入力テキストを拡張します。

- テキスト分類のタスクにおいては、文書の最初に```<s>```、最後に```<e>```が追加されます。
- 自然言語推論のタスクにおいては、テキストの境界に```$```が挿入されます。

![](./Figure/gpt-2-autoregression-2.gif)


### Huggingface transformerを使う


In [1]:
from transformers import pipeline
#!pip install sentencepiece
#!pip install protobuf
generator = pipeline("text-generation", model="abeja/gpt2-large-japanese")


config.json:   0%|          | 0.00/974 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/784k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/153 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
generated = generator(
    "東北大学は",
    max_length=100,
    do_sample=True,
    num_return_sequences=3,
    top_p=0.95,
    top_k=50,
    pad_token_id=3
)
print(*generated, sep="\n")

{'generated_text': '東北大学は、学術的な情報や、学生・院生に関する情報などを掲載した学生・院生情報誌の発行や、留学生および外国人研究者向けの日本語学習教材の開発を予定しています。 文部科学省は、平成25年度から3年間を計画期間とする、大学国際化推進プログラムの実施に向けた具体的な検討を始めた。 平成25年度と平成26年度の2年間にわたる大学国際化推進プログラムは、大学発の産学連携や国際交流への取り組みを促進させるとともに、大学'}
{'generated_text': '東北大学は「学生の本分は学業」であることは勿論であるが、学問と研究は同時に行わなければ達成できない。研究は単なる自己満足ではなく、社会に有益な研究が不可欠である。このような基本姿勢をもって、各年次における教育活動が行われており、本学の教職員は大学教育の責務と自覚をもって、大学教育の改善に積極的に努めることを学則で定めております。 本学は、社会の要請及び大学の存在意義に応えるために'}
{'generated_text': '東北大学は、東日本大震災での教訓を踏まえ、地震による建物被害を防ぐため、建築確認等の制度を抜本的に改正して、平成28年10月に新建築基準法が施行されました。新建築基準法に基づく確認調査・評価制度では、建築物の耐震性や設計・施工等のあり方を客観的な手法で調査・評価し、その結果として必要な対策を講ずることを定めています。 宮城県建築士事務所協会では、建築士事務所協会の会員の皆さまの自主的な建築基準'}
